In [108]:
import requests
import requests_cache
from matplotlib import pyplot as plt
import lxml.html
import pandas as pd
import re
import json 
import lxml.html as lx
import urllib
from urllib2 import Request, urlopen, HTTPError
from urlparse import urlunparse, urlparse, urljoin
import numpy
import plotly.plotly as py
import plotly.figure_factory as ff
import math
from bs4 import BeautifulSoup

In [109]:
def fetch_page(url):
    response = requests.get(url)
    response.raise_for_status()
    return response

url="https://www.zillow.com/browse/homes/ca/"
requests_cache.install_cache("cache")
page = fetch_page(url)
html = lx.fromstring(page.text)
county_html = html.xpath('//main/div/div/ul/li/a')
county = [i.text_content() for i in county_html]
county_name = pd.DataFrame({"RegionName":[" ".join(k) for k in [i.split(" ")[:-2] for i in county]]})

In [110]:
#Import Housing Data
import os.path

def dataframe(i = 0):
    
    """Read datasets
    Please change the file path to fit to your computer
    Input:the name of dataset //dataname   
    """
    
    file_path = "/Users/HUI/Dropbox/16_UCDavis/17W_STA141B/05_Project/Housing_Data"
    file_name = ["County_MedianSoldPrice_AllHomes.csv","County_MedianListingPrice_AllHomes.csv","County_MedianListingPrice_1Bedroom.csv",
                 "County_MedianListingPrice_2Bedroom.csv","County_MedianListingPrice_3Bedroom.csv","County_MedianListingPrice_4Bedroom.csv",
                 "County_MedianListingPrice_5BedroomOrMore.csv","County_MedianListingPrice_CondoCoop.csv","County_MedianListingPrice_DuplexTriplex.csv",
                 "County_MedianListingPrice_Sfr.csv"]
    dataframe = pd.read_csv(os.path.join(file_path,file_name[i]),header=0)
    CA_dataframe = dataframe[dataframe["State"]== "CA"]
    CA_dataframe = CA_dataframe.reset_index(drop=True)
    return(CA_dataframe)

###name of eachdata
dataname=["sold","list","one","two","thre","four","five","cond","trip","sfr"]

###save all of dataset to CA dict.
CA = {i: dataframe(dataname.index(i)) for i in dataname}

In [111]:
#All Home Sold time-series data 1996-2016
CA["sold"] = pd.merge(pd.DataFrame(county_name),CA["sold"], how='left', on=['RegionName'])
CA_sold_t = CA["sold"].transpose()
CA_sold_t = CA_sold_t.rename(columns=lambda i: CA["sold"]["RegionName"][i])[7:]
date = pd.date_range('1996-04', '2016-07', freq='M')
CA_sold_t['date'] = date

#sold price time-series data
CA_sold_t = CA_sold_t.set_index('date')
CA_sold_t.head()


,Los Angeles,Orange,San Diego,Riverside,San Bernardino,Santa Clara,Alameda,Sacramento,Contra Costa,Fresno,...,Glenn,Plumas,Colusa,Mariposa,Inyo,Trinity,Mono,Modoc,Sierra,Alpine
date,,,,,,,,,,,,,,,,,,,,,
1996-04-30,162225,191525,158225,114550,100250,234325,196275,120425,192750,94850,...,67000,NaN,NaN,NaN,121471,53000,77500,NaN,NaN,NaN
1996-05-31,162925,197550,158125,112500,101700,238575,196900,121675,203000,95475,...,65823.5,NaN,97230.8,115000,NaN,NaN,78970.6,NaN,NaN,NaN
1996-06-30,164500,197875,158925,115000,103450,238875,202225,122175,209050,95475,...,NaN,NaN,NaN,NaN,119077,51950,NaN,NaN,NaN,NaN
1996-07-31,164550,198050,158700,113975,101850,241875,203700,122175,207150,96550,...,NaN,NaN,NaN,94230.8,112882,NaN,81538.5,NaN,NaN,NaN
1996-08-31,163300,199300,157300,113225,101650,241200,202250,121075,202500,96625,...,NaN,80000,NaN,NaN,NaN,82300,98470.6,NaN,NaN,NaN


In [112]:
#Different type of house average yr 2015/2016
def aver(i="one"):
    data = CA[i]
    CA_2015 = data.iloc[:,[k.__contains__("2015") for k in data.keys()]]
    CA_2016 = data.iloc[:,[k.__contains__("2016") for k in data.keys()]]
    CA_mean = pd.DataFrame({i+"_15":CA_2015.mean(1),i+"_16":CA_2016.mean(1)})
    CA_ave_name = pd.DataFrame({"RegionName":data["RegionName"]})
    CA_ave = pd.concat([CA_ave_name,CA_mean],axis=1)
    return CA_ave
CA_ave=[aver(i) for i in dataname]


#average price in yr 25/26 for different type of house/and all homes
diff_ave = reduce(lambda left,right: pd.merge(left,right,how="left",on='RegionName'), CA_ave)
diff_ave.head()

,RegionName,sold_15,sold_16,list_15,list_16,one_15,one_16,two_15,two_16,thre_15,...,four_15,four_16,five_15,five_16,cond_15,cond_16,trip_15,trip_16,sfr_15,sfr_16
0,Los Angeles,498737.500000,518508.333333,537666.666667,587328.666667,363468.333333,409424.500000,433116.583333,469545.750000,497107.916667,...,659808.333333,701394.000000,1.038750e+06,1.176233e+06,452570.833333,488478.750000,664705.500000,7.312750e+05,576524.500000,629662.333333
1,Orange,601020.833333,623150.000000,675932.083333,705890.000000,279250.000000,303945.833333,417687.500000,441220.833333,639658.333333,...,841700.000000,883811.583333,1.382648e+06,1.439552e+06,422483.333333,446204.166667,924108.583333,1.120979e+06,787609.750000,820343.541667
2,San Diego,472187.500000,489250.000000,538650.000000,575525.000000,276475.000000,319328.625000,360778.166667,396704.083333,494828.166667,...,684390.666667,716899.958333,1.093432e+06,1.184135e+06,376438.583333,410828.333333,678408.333333,7.657750e+05,609515.250000,641470.583333
3,Riverside,309068.750000,320437.500000,344257.500000,361732.458333,145399.875000,145940.750000,227945.750000,237962.041667,327233.333333,...,394170.750000,413642.833333,4.613150e+05,4.925966e+05,273656.666667,280891.666667,352866.583333,3.872125e+05,356255.250000,376051.708333
4,San Bernardino,264327.083333,272666.666667,271312.500000,291733.333333,NaN,NaN,172779.083333,185468.166667,255624.833333,...,367520.833333,385991.166667,4.659978e+05,4.931395e+05,259899.916667,271735.416667,295638.125000,3.068167e+05,272363.916667,293408.333333


In [113]:
file_path = "/Users/HUI/Dropbox/16_UCDavis/17W_STA141B/05_Project/"
def readdata(file_name):
             #,mergedata=diff_ave):

    dataframe = pd.read_csv(os.path.join(file_path,file_name),header=0)
    
    return(dataframe)

Unemp_frame = readdata("Unemploy_Rate.csv")
Unemp_frame["County"] = [" ".join(i.split(" ")[:-1]) for i in Unemp_frame["County"]]
Unemp_frame.head()

,County,Unemploy_Rate
0,Alameda,4.3
1,Alpine,5.9
2,Amador,6.3
3,Butte,6.9
4,Calaveras,6.2


In [276]:
###ZipCode housing price
def dataframe_zip(i = 0):
    
    """Read datasets
    Please change the file path to fit to your computer
    Input:the name of dataset //dataname   
    """
    
    file_path = "/Users/HUI/Dropbox/16_UCDavis/17W_STA141B/05_Project/Housing_Data"
    file_name = ["Zip_MedianSoldPrice_AllHomes.csv","Zip_MedianListingPrice_AllHomes.csv","Zip_MedianListingPrice_1Bedroom.csv",
                 "Zip_MedianListingPrice_2Bedroom.csv","Zip_MedianListingPrice_3Bedroom.csv","Zip_MedianListingPrice_4Bedroom.csv",
                 "Zip_MedianListingPrice_5BedroomOrMore.csv","Zip_MedianListingPrice_CondoCoop.csv","Zip_MedianListingPrice_DuplexTriplex.csv",
                 "Zip_MedianListingPrice_Sfr.csv"]
    dataframe = pd.read_csv(os.path.join(file_path,file_name[i]),header=0)
    CA_dataframe = dataframe[dataframe["State"]== "CA"]
    CA_dataframe = CA_dataframe.reset_index(drop=True)
    return(CA_dataframe)

###name of eachdata
dataname=["sold","list","one","two","thre","four","five","cond","trip","sfr"]

###save all of dataset to CA dict.
CA_zip = {i: dataframe_zip(dataname.index(i)) for i in dataname}
CA_zip['sold']

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06
0,97564,94109,San Francisco,CA,San Francisco,San Francisco,8,259250.0000,252000.0000,263950.0000,...,1123750.0,1080500.0,9.693500e+05,1.070900e+06,NaN,NaN,8.990000e+05,1.017235e+06,9.798500e+05,NaN
1,96107,90250,Hawthorne,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,16,155850.0000,157750.0000,165725.0000,...,510550.0,487775.0,NaN,5.290000e+05,4.852941e+05,441000.0,4.716500e+05,5.026000e+05,5.015000e+05,5.145000e+05
2,96027,90046,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,26,288825.0000,285925.0000,309250.0000,...,1020750.0,1017500.0,8.757500e+05,8.246500e+05,7.674750e+05,NaN,1.399450e+06,1.193000e+06,NaN,9.791000e+05
3,97771,94565,Pittsburg,CA,San Francisco,Contra Costa,27,123800.0000,125750.0000,119225.0000,...,327250.0,326800.0,3.276000e+05,3.396250e+05,3.222250e+05,325000.0,3.372500e+05,3.502500e+05,3.580000e+05,3.551250e+05
4,97711,94501,Alameda,CA,San Francisco,Alameda,50,191650.0000,193950.0000,193700.0000,...,742100.0,754000.0,7.666500e+05,7.907500e+05,7.465000e+05,NaN,8.226923e+05,8.245294e+05,7.963000e+05,7.536000e+05
5,97565,94110,San Francisco,CA,San Francisco,San Francisco,55,220900.0000,224575.0000,237125.0000,...,1195500.0,1209000.0,1.215500e+06,1.189500e+06,1.210500e+06,1225000.0,1.268000e+06,1.334500e+06,1.356500e+06,1.391500e+06
6,96015,90034,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,57,210000.8235,212501.3000,199250.7000,...,846250.0,861850.0,NaN,7.980769e+05,7.597059e+05,NaN,1.102077e+06,1.115059e+06,9.587000e+05,8.945000e+05
7,96025,90044,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,61,113950.0000,121000.0000,120650.0000,...,316250.0,NaN,3.592308e+05,3.368235e+05,3.034500e+05,NaN,3.083077e+05,NaN,3.503077e+05,3.629412e+05
8,96193,90650,Norwalk,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,66,137650.1500,139450.0000,138600.0000,...,396500.0,398500.0,3.945000e+05,3.861500e+05,3.863500e+05,391000.0,4.017500e+05,4.027500e+05,4.015000e+05,4.075000e+05
9,96239,90805,Long Beach,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,69,121900.0000,121250.0000,122900.0000,...,370125.0,360750.0,3.716500e+05,3.771000e+05,3.782500e+05,380750.0,3.785000e+05,3.815000e+05,3.845000e+05,3.897000e+05


In [307]:
#Different type of house average yr 2015/2016 for zip code
def aver_zip(i="one"):
    data = CA_zip[i]
    CA_2015 = data.iloc[:,[k.__contains__("2015") for k in data.keys()]]
    CA_2016 = data.iloc[:,[k.__contains__("2016") for k in data.keys()]]
    CA_mean = pd.DataFrame({i+"_15":CA_2015.mean(1),i+"_16":CA_2016.mean(1)})
    CA_ave_name = pd.DataFrame({"RegionName":data["RegionName"],"CountyName":data["CountyName"]})
    CA_ave_zip = pd.concat([CA_ave_name,CA_mean],axis=1)
    return CA_ave_zip

CA_ave_zip=[aver_zip(i) for i in dataname]


#average price in yr 25/26 for different type of house/and all homes
diff_ave_zip = reduce(lambda left,right:pd.merge(left, right,how='left', on=['RegionName','CountyName']) , CA_ave_zip)
diff_ave_zip

,CountyName,RegionName,sold_15,sold_16,list_15,list_16,one_15,one_16,two_15,two_16,...,four_15,four_16,five_15,five_16,cond_15,cond_16,trip_15,trip_16,sfr_15,sfr_16
0,San Francisco,94109,9.949333e+05,9.653618e+05,1.155800e+06,1.096792e+06,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.195000e+06,1.046583e+06,NaN,NaN,NaN,NaN
1,Los Angeles,90250,5.226458e+05,4.860907e+05,4.796205e+05,5.443083e+05,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.780625e+05,5.364279e+05
2,Los Angeles,90046,9.369148e+05,1.084756e+06,1.220125e+06,1.371871e+06,NaN,NaN,NaN,NaN,...,2.235417e+06,2.284125e+06,NaN,NaN,NaN,6.584330e+05,NaN,NaN,1.623625e+06,1.814831e+06
3,Contra Costa,94565,3.185021e+05,3.413083e+05,3.400394e+05,3.852135e+05,NaN,NaN,NaN,NaN,...,3.879000e+05,4.395845e+05,NaN,NaN,NaN,NaN,NaN,NaN,3.434932e+05,3.869069e+05
4,Alameda,94501,7.456667e+05,7.887243e+05,NaN,8.449757e+05,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.029506e+05,8.911818e+05
5,San Francisco,94110,1.208667e+06,1.297667e+06,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.311455e+06
6,Los Angeles,90034,7.979840e+05,9.660083e+05,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.451167e+05,1.138292e+06
7,Los Angeles,90044,3.220523e+05,3.312516e+05,3.035449e+05,3.413329e+05,NaN,NaN,292062.083333,319105.833333,...,NaN,NaN,4.929444e+05,4.961958e+05,NaN,NaN,410062.458333,4.726242e+05,3.061708e+05,3.425291e+05
8,Los Angeles,90650,3.800458e+05,3.984750e+05,3.760083e+05,4.124082e+05,NaN,NaN,343837.000000,368516.625000,...,4.202333e+05,4.481833e+05,NaN,NaN,NaN,NaN,NaN,NaN,3.872250e+05,4.202000e+05
9,Los Angeles,90805,3.529583e+05,3.822000e+05,3.431417e+05,3.824249e+05,NaN,NaN,321225.000000,362066.583333,...,NaN,NaN,NaN,NaN,NaN,NaN,473125.000000,5.113167e+05,3.508495e+05,3.873750e+05


In [308]:
#conbine zipcode data with school rank#
file_path = "/Users/HUI/Dropbox/16_UCDavis/17W_STA141B/05_Project"
school = pd.read_excel(os.path.join(file_path,"school.xlsx"),skiprows = [1,2,3], header = 1)
school.head()
school_rank = school.iloc[:,[0,1,3,5,6,13,14,15,17]]
school_rank.columns = ['Rank2016','District','City','RegionName','County','Score2016','Score2015','Rank2015','SchoolDiggerRating']
school_rank
zip_school_home= pd.merge(school_rank,diff_ave_zip,on="RegionName")
zip_school_home

zip_school_home.to_csv("/Users/HUI/Dropbox/16_UCDavis/17W_STA141B/05_Project/zip_school_price.csv")


In [309]:
Pop_frame = readdata("population.csv")
Pop_frame = Pop_frame[["County","Population"]]
Pop_frame.head()

,County,Population
0,Alameda,1638215
1,Alpine,1110
2,Amador,37001
3,Butte,225411
4,Calaveras,44828


In [118]:
Unem_diff=pd.merge(diff_ave,Unemp_frame,how="left",left_on = "RegionName",right_on = "County")
Unem_diff.head()

,RegionName,sold_15,sold_16,list_15,list_16,one_15,one_16,two_15,two_16,thre_15,...,five_15,five_16,cond_15,cond_16,trip_15,trip_16,sfr_15,sfr_16,County,Unemploy_Rate
0,Los Angeles,498737.500000,518508.333333,537666.666667,587328.666667,363468.333333,409424.500000,433116.583333,469545.750000,497107.916667,...,1.038750e+06,1.176233e+06,452570.833333,488478.750000,664705.500000,7.312750e+05,576524.500000,629662.333333,Los Angeles,5.7
1,Orange,601020.833333,623150.000000,675932.083333,705890.000000,279250.000000,303945.833333,417687.500000,441220.833333,639658.333333,...,1.382648e+06,1.439552e+06,422483.333333,446204.166667,924108.583333,1.120979e+06,787609.750000,820343.541667,Orange,4.1
2,San Diego,472187.500000,489250.000000,538650.000000,575525.000000,276475.000000,319328.625000,360778.166667,396704.083333,494828.166667,...,1.093432e+06,1.184135e+06,376438.583333,410828.333333,678408.333333,7.657750e+05,609515.250000,641470.583333,San Diego,4.8
3,Riverside,309068.750000,320437.500000,344257.500000,361732.458333,145399.875000,145940.750000,227945.750000,237962.041667,327233.333333,...,4.613150e+05,4.925966e+05,273656.666667,280891.666667,352866.583333,3.872125e+05,356255.250000,376051.708333,Riverside,6.0
4,San Bernardino,264327.083333,272666.666667,271312.500000,291733.333333,NaN,NaN,172779.083333,185468.166667,255624.833333,...,4.659978e+05,4.931395e+05,259899.916667,271735.416667,295638.125000,3.068167e+05,272363.916667,293408.333333,San Bernardino,5.7


In [119]:
Unem_diff_pop=pd.merge(Unem_diff,Pop_frame,how="left",left_on = "RegionName",right_on = "County")
Unem_diff_pop.head()

,RegionName,sold_15,sold_16,list_15,list_16,one_15,one_16,two_15,two_16,thre_15,...,cond_15,cond_16,trip_15,trip_16,sfr_15,sfr_16,County_x,Unemploy_Rate,County_y,Population
0,Los Angeles,498737.500000,518508.333333,537666.666667,587328.666667,363468.333333,409424.500000,433116.583333,469545.750000,497107.916667,...,452570.833333,488478.750000,664705.500000,7.312750e+05,576524.500000,629662.333333,Los Angeles,5.7,Los Angeles,10170292
1,Orange,601020.833333,623150.000000,675932.083333,705890.000000,279250.000000,303945.833333,417687.500000,441220.833333,639658.333333,...,422483.333333,446204.166667,924108.583333,1.120979e+06,787609.750000,820343.541667,Orange,4.1,Orange,3169776
2,San Diego,472187.500000,489250.000000,538650.000000,575525.000000,276475.000000,319328.625000,360778.166667,396704.083333,494828.166667,...,376438.583333,410828.333333,678408.333333,7.657750e+05,609515.250000,641470.583333,San Diego,4.8,San Diego,3299521
3,Riverside,309068.750000,320437.500000,344257.500000,361732.458333,145399.875000,145940.750000,227945.750000,237962.041667,327233.333333,...,273656.666667,280891.666667,352866.583333,3.872125e+05,356255.250000,376051.708333,Riverside,6.0,Riverside,2361026
4,San Bernardino,264327.083333,272666.666667,271312.500000,291733.333333,NaN,NaN,172779.083333,185468.166667,255624.833333,...,259899.916667,271735.416667,295638.125000,3.068167e+05,272363.916667,293408.333333,San Bernardino,5.7,San Bernardino,2128133


In [120]:
crim = readdata("crime.csv")
Unme_diff_pop_crim = pd.merge(Unem_diff_pop,crim.iloc[:,1:],how="left", on = "RegionName")
Unme_diff_pop_crim.head()

,RegionName,sold_15,sold_16,list_15,list_16,one_15,one_16,two_15,two_16,thre_15,...,cond_16,trip_15,trip_16,sfr_15,sfr_16,County_x,Unemploy_Rate,County_y,Population,CrimeCases
0,Los Angeles,498737.500000,518508.333333,537666.666667,587328.666667,363468.333333,409424.500000,433116.583333,469545.750000,497107.916667,...,488478.750000,664705.500000,7.312750e+05,576524.500000,629662.333333,Los Angeles,5.7,Los Angeles,10170292,43163.0
1,Orange,601020.833333,623150.000000,675932.083333,705890.000000,279250.000000,303945.833333,417687.500000,441220.833333,639658.333333,...,446204.166667,924108.583333,1.120979e+06,787609.750000,820343.541667,Orange,4.1,Orange,3169776,2967.0
2,San Diego,472187.500000,489250.000000,538650.000000,575525.000000,276475.000000,319328.625000,360778.166667,396704.083333,494828.166667,...,410828.333333,678408.333333,7.657750e+05,609515.250000,641470.583333,San Diego,4.8,San Diego,3299521,11931.0
3,Riverside,309068.750000,320437.500000,344257.500000,361732.458333,145399.875000,145940.750000,227945.750000,237962.041667,327233.333333,...,280891.666667,352866.583333,3.872125e+05,356255.250000,376051.708333,Riverside,6.0,Riverside,2361026,17148.0
4,San Bernardino,264327.083333,272666.666667,271312.500000,291733.333333,NaN,NaN,172779.083333,185468.166667,255624.833333,...,271735.416667,295638.125000,3.068167e+05,272363.916667,293408.333333,San Bernardino,5.7,San Bernardino,2128133,12956.0


In [121]:
Unme_diff_pop_crim.keys()

Index([u'RegionName', u'sold_15', u'sold_16', u'list_15', u'list_16',
       u'one_15', u'one_16', u'two_15', u'two_16', u'thre_15', u'thre_16',
       u'four_15', u'four_16', u'five_15', u'five_16', u'cond_15', u'cond_16',
       u'trip_15', u'trip_16', u'sfr_15', u'sfr_16', u'County_x',
       u'Unemploy_Rate', u'County_y', u'Population', u'CrimeCases'],
      dtype='object')

In [122]:
Unme_diff_pop_crim.drop(["County_x",'County_y'],axis=1, inplace=True)

In [123]:
Unme_diff_pop_crim.keys()

Index([u'RegionName', u'sold_15', u'sold_16', u'list_15', u'list_16',
       u'one_15', u'one_16', u'two_15', u'two_16', u'thre_15', u'thre_16',
       u'four_15', u'four_16', u'five_15', u'five_16', u'cond_15', u'cond_16',
       u'trip_15', u'trip_16', u'sfr_15', u'sfr_16', u'Unemploy_Rate',
       u'Population', u'CrimeCases'],
      dtype='object')

In [124]:
rank = readdata("county_all_rank.csv")
all_rank = pd.merge(Unme_diff_pop_crim,rank,how="left", left_on = "RegionName",right_on = "County")
all_rank.drop(['County'],axis=1, inplace=True)
all_rank.keys()

Index([u'RegionName', u'sold_15', u'sold_16', u'list_15', u'list_16',
       u'one_15', u'one_16', u'two_15', u'two_16', u'thre_15', u'thre_16',
       u'four_15', u'four_16', u'five_15', u'five_16', u'cond_15', u'cond_16',
       u'trip_15', u'trip_16', u'sfr_15', u'sfr_16', u'Unemploy_Rate',
       u'Population', u'CrimeCases', u'Score2016', u'Score2015', u'Rank2016',
       u'Highest Rank City 2016', u'Zip2016', u'Rank2015',
       u'Highest Rank City 2015', u'Zip2015', u'Number Students',
       u'Number of Ranked Elementary Schools',
       u'Number of Ranked Middle Schools', u'Number of Ranked High Schools'],
      dtype='object')

In [125]:
income = readdata("income.csv")
#income = income.drop(income.iloc[:,:3])
all_income = pd.merge(all_rank,income.iloc[:,3:],how="left", left_on = "RegionName",right_on = "county")
all_income.drop(['county'],axis=1, inplace=True)


In [126]:
CA_sold_t.to_csv("/Users/HUI/Dropbox/16_UCDavis/17W_STA141B/05_Project/CA_sold_t.csv")
diff_ave.to_csv("/Users/HUI/Dropbox/16_UCDavis/17W_STA141B/05_Project/diff_ave.csv")

In [127]:
all_income.to_csv("/Users/HUI/Dropbox/16_UCDavis/17W_STA141B/05_Project/all_data.csv")

In [341]:
all_data = all_income

In [363]:

CA_sold_t.resample('A')


DatetimeIndexResampler [freq=<YearEnd: month=12>, axis=0, closed=right, label=right, convention=start, base=0]


In [ ]:
###time-series data combind

In [22]:
import plotly
import plotly.plotly as py
import plotly.tools as tls
import plotly.graph_objs as go

In [23]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Scatter, Figure, Layout
init_notebook_mode(connected=True)
import cufflinks as cf
iplot(CA_sold_t.iplot(asFigure=True,
                               kind='line',xTitle='Dates',yTitle='Sold Price',title='Time-Series Solde Price for County'))


In [31]:
import plotly.plotly as py
import plotly.graph_objs as go 

from datetime import datetime
import pandas_datareader as web

upper_bound0 = go.Scatter(
    name='Upper Bound_med',
    x=CA_sold_t.index,
    y=CA_sold_t.mean(axis=1)+CA_sold_t.std(axis=1),
    mode='lines',
    marker=dict(color="444"),
    line=dict(width=0),
    fillcolor='rgba(68, 68, 68, 0.3)',
    fill='tonexty' )

trace0 = go.Scatter(
    x=CA_sold_t.index,
    y=CA_sold_t.median(axis=1),
    fillcolor='rgba(68, 68, 68, 0.3)',
    name = "median",
    fill='tonexty'
)

trace1 = go.Scatter(
    x=CA_sold_t.index,
    y=CA_sold_t.mean(axis=1),
    line=dict(color='rgb(31, 119, 180)'),
    fillcolor='rgba(68, 68, 68, 0.3)',
    name = "mean",
    fill='tonexty'
)

lower_bound0 = go.Scatter(
    name='Lower Bound_med',
    x=CA_sold_t.index,
    y=CA_sold_t.mean(axis=1)-CA_sold_t.std(axis=1),
    marker=dict(color="444"),
    line=dict(width=0),
    mode='lines' )


data = [lower_bound0,trace0,trace1,upper_bound0]



layout = dict(
    title='Sold Price from 1996 to 2016 in Calforina',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=6,
                     label='6m',
                     step='month',
                     stepmode='backward'),
                dict(count=1,
                    label='1y',
                    step='year',
                    stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(),
        type='date'
    )
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)


In [45]:
###medain sales price compare with US and CA in 2015
median_us=296400
all_data["sold_15"].median()



trace0 = go.Bar(
    x=[median_us, all_data["sold_15"].mean()],
    y=['US ','California'],
    orientation = 'h',
    marker=dict(
        color=['rgba(204,204,204,1)','rgba(222,45,38,0.8)']),
)

data = [trace0]

layout = go.Layout(
    title='Medidan Housing Price between US and California',
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='color-bar')


In [129]:
import urllib
import ast
import HTMLParser
import string
import json

In [130]:
#url = 'http://catalog.civicdashboards.com/dataset/ce409ee1-5128-4b66-aa3e-957dbd4de8ba/resource/6f805645-0836-478d-b168-c1f72d53b4f3/download/5faf934922fd4748a843d55990297d9ftemp.geojson'
california_data = json.load(open('/Users/HUI/Dropbox/16_UCDavis/17W_STA141B/05_Project/california.json'))


In [131]:
import folium
import json
from branca.utilities import _locations_mirror
from folium.features import *
from folium.features import _locations_tolist

In [132]:
class MultiPolygon(MacroElement):
    """
    !! This is hacked from folium.features.MultiPolyLine !! 
    
    """
    def __init__(self, locations, color=None, weight=None,
                 opacity=None, latlon=True, popup=None):
        super(MultiPolygon, self).__init__()
        self._name = 'MultiPolygon'
        self.data = (_locations_mirror(locations) if not latlon else
                     _locations_tolist(locations))
        self.color = color
        self.weight = weight
        self.opacity = opacity
        if isinstance(popup, text_type) or isinstance(popup, binary_type):
            self.add_children(Popup(popup))
        elif popup is not None:
            self.add_children(popup)

        self._template = Template(u"""
            {% macro script(this, kwargs) %}
                var {{this.get_name()}} = L.multiPolygon(
                    {{this.data}},
                    {
                        {% if this.color != None %}color: '{{ this.color }}',{% endif %}
                        {% if this.weight != None %}weight: {{ this.weight }},{% endif %}
                        {% if this.opacity != None %}opacity: {{ this.opacity }},{% endif %}
                        });
                {{this._parent.get_name()}}.addLayer({{this.get_name()}});
            {% endmacro %}
            """)  # noqa

    def _get_self_bounds(self):
        """Computes the bounds of the object itself (not including it's children)
        in the form [[lat_min, lon_min], [lat_max, lon_max]]
        """
        bounds = [[None, None], [None, None]]
        for point in iter_points(self.data):
            bounds = [
                [
                    none_min(bounds[0][0], point[0]),
                    none_min(bounds[0][1], point[1]),
                ],
                [
                    none_max(bounds[1][0], point[0]),
                    none_max(bounds[1][1], point[1]),
                ],
            ]
        return bounds

In [133]:
all_data

,RegionName,sold_15,sold_16,list_15,list_16,one_15,one_16,two_15,two_16,thre_15,...,Highest Rank City 2016,Zip2016,Rank2015,Highest Rank City 2015,Zip2015,Number Students,Number of Ranked Elementary Schools,Number of Ranked Middle Schools,Number of Ranked High Schools,total_income
0,Los Angeles,4.987375e+05,5.185083e+05,5.376667e+05,5.873287e+05,363468.333333,409424.500000,4.331166e+05,4.695458e+05,4.971079e+05,...,Acton,90017,9.0,Acton,90017,1534702.0,1275.0,502.0,469.0,3263069
1,Orange,6.010208e+05,6.231500e+05,6.759321e+05,7.058900e+05,279250.000000,303945.833333,4.176875e+05,4.412208e+05,6.396583e+05,...,Anaheim,90620,31.0,Anaheim,90620,496719.0,385.0,113.0,100.0,1009353
2,San Diego,4.721875e+05,4.892500e+05,5.386500e+05,5.755250e+05,276475.000000,319328.625000,3.607782e+05,3.967041e+05,4.948282e+05,...,Alpine,91901,10.0,Alpine,91901,502352.0,434.0,188.0,145.0,1094157
3,Riverside,3.090688e+05,3.204375e+05,3.442575e+05,3.617325e+05,145399.875000,145940.750000,2.279458e+05,2.379620e+05,3.272333e+05,...,Banning,92220,123.0,Banning,92220,425225.0,284.0,104.0,105.0,699232
4,San Bernardino,2.643271e+05,2.726667e+05,2.713125e+05,2.917333e+05,NaN,NaN,1.727791e+05,1.854682e+05,2.556248e+05,...,Adelanto,91701,115.0,Adelanto,91701,409680.0,336.0,125.0,100.0,614325
5,Santa Clara,7.992500e+05,8.563750e+05,8.225505e+05,8.882302e+05,441718.666667,467069.833333,5.505682e+05,6.242908e+05,7.652370e+05,...,Campbell,94024,4.0,Campbell,94024,275810.0,248.0,93.0,75.0,621463
6,Alameda,6.147958e+05,6.618083e+05,6.279979e+05,6.877866e+05,350766.666667,398109.916667,4.469397e+05,5.095338e+05,6.270787e+05,...,Alameda,94501,44.0,Alameda,94501,224453.0,219.0,87.0,83.0,558907
7,Sacramento,2.751083e+05,2.914417e+05,3.066311e+05,3.241530e+05,NaN,163918.250000,NaN,2.231291e+05,2.737263e+05,...,Antelope,95608,209.0,Antelope,95608,237811.0,221.0,101.0,81.0,522596
8,Contra Costa,4.738750e+05,5.033750e+05,5.213989e+05,5.617979e+05,247643.750000,282395.375000,3.626432e+05,4.009770e+05,4.661188e+05,...,Antioch,94509,2.0,Antioch,94509,174213.0,148.0,54.0,55.0,384646
9,Fresno,2.213333e+05,2.234167e+05,2.538533e+05,2.573312e+05,NaN,NaN,1.326749e+05,1.414245e+05,2.247429e+05,...,Auberry,91398,35.0,Auberry,91398,199340.0,186.0,80.0,65.0,296305


In [134]:
geo_dict = {}
for x in range(len(california_data['features'])):
    # I ignore the last eleven characters in the name since the geojson file includes ' County, CO' in the county names and the population data does not
    name = california_data['features'][x]['properties']['name'][:-11]
    if name in all_data['RegionName'].unique():
        geo_dict[name] = california_data['features'][x]
    else:
        print 'not in: ', name

ser = pd.Series(geo_dict.values(), index = geo_dict.keys())
ser.name = 'coordinates'
all_data_cord = all_data.join(ser, on='RegionName')
all_data_sub = all_data_cord[['RegionName','sold_15','sold_16','list_15','list_16','Score2016','Score2015',"coordinates"]]
all_data_sub.head()

,RegionName,sold_15,sold_16,list_15,list_16,Score2016,Score2015,coordinates
0,Los Angeles,498737.500000,518508.333333,537666.666667,587328.666667,0.575243,0.569822,"{u'geometry': {u'type': u'MultiPolygon', u'coo..."
1,Orange,601020.833333,623150.000000,675932.083333,705890.000000,0.674438,0.682548,"{u'geometry': {u'type': u'MultiPolygon', u'coo..."
2,San Diego,472187.500000,489250.000000,538650.000000,575525.000000,0.617286,0.625467,"{u'geometry': {u'type': u'MultiPolygon', u'coo..."
3,Riverside,309068.750000,320437.500000,344257.500000,361732.458333,0.410757,0.411668,"{u'geometry': {u'type': u'MultiPolygon', u'coo..."
4,San Bernardino,264327.083333,272666.666667,271312.500000,291733.333333,0.434529,0.442580,"{u'geometry': {u'type': u'MultiPolygon', u'coo..."


In [135]:
colors = ['#ffffe0','#fffddb','#fffad7','#fff7d1','#fff5cd','#fff2c8',
          '#fff0c4','#ffedbf','#ffebba','#ffe9b7','#ffe5b2','#ffe3af',
          '#ffe0ab','#ffdda7','#ffdba4','#ffd9a0','#ffd69c','#ffd399',
          '#ffd196','#ffcd93','#ffca90','#ffc88d','#ffc58a','#ffc288',
          '#ffbf86','#ffbd83','#ffb981','#ffb67f','#ffb47d','#ffb17b',
          '#ffad79','#ffaa77','#ffa775','#ffa474','#ffa172','#ff9e70',
          '#ff9b6f','#ff986e','#ff956c','#fe916b','#fe8f6a','#fd8b69',
          '#fc8868','#fb8567','#fa8266','#f98065','#f87d64','#f77a63',
          '#f67862','#f57562','#f37261','#f37060','#f16c5f','#f0695e',
          '#ee665d','#ed645c','#ec615b','#ea5e5b','#e85b59','#e75859',
          '#e55658','#e45356','#e35056','#e14d54','#df4a53','#dd4852',
          '#db4551','#d9434f','#d8404e','#d53d4d','#d43b4b','#d2384a',
          '#cf3548','#cd3346','#cc3045','#ca2e43','#c72b42','#c52940',
          '#c2263d','#c0233c','#be213a','#bb1e37','#ba1c35','#b71933',
          '#b41731','#b2152e','#b0122c','#ac1029','#aa0e27','#a70b24',
          '#a40921','#a2071f','#a0051c','#9d0419','#990215','#970212',
          '#94010e','#91000a','#8e0006','#8b0000', '#8b0000']

scl = dict(zip(range(0, 101), colors))
colorscl = [[i * .01, v] for i,v in enumerate(colors)]

In [181]:
def get_scl(obj,number):
    """
    assign color
    """
    if pd.isnull(obj):
        return scl[0] 
    else:
        frac = int(obj/number)
        return scl[frac]

all_data_sub['color'] = all_data_sub['sold_15'].apply(get_scl,number = 12500)
all_data_sub.head()


/Users/HUI/miniconda2/lib/python2.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,RegionName,sold_15,sold_16,list_15,list_16,Score2016,Score2015,coordinates,color
0,Los Angeles,498737.500000,518508.333333,537666.666667,587328.666667,0.575243,0.569822,"{u'geometry': {u'type': u'MultiPolygon', u'coo...",#fe916b
1,Orange,601020.833333,623150.000000,675932.083333,705890.000000,0.674438,0.682548,"{u'geometry': {u'type': u'MultiPolygon', u'coo...",#f67862
2,San Diego,472187.500000,489250.000000,538650.000000,575525.000000,0.617286,0.625467,"{u'geometry': {u'type': u'MultiPolygon', u'coo...",#ff986e
3,Riverside,309068.750000,320437.500000,344257.500000,361732.458333,0.410757,0.411668,"{u'geometry': {u'type': u'MultiPolygon', u'coo...",#ffbf86
4,San Bernardino,264327.083333,272666.666667,271312.500000,291733.333333,0.434529,0.442580,"{u'geometry': {u'type': u'MultiPolygon', u'coo...",#ffc88d


In [182]:
requests_cache.install_cache("cache") 
url = requests.get("https://en.wikipedia.org/wiki/User:Michael_J/County_table") 
soup = BeautifulSoup(url.text)

sort = []
state = []
FIPS = []
county = []
county_seat = []
population = []
land_area_km = []
land_area_ml = []
water_area_km = []
water_area_ml = []
total_area_km = []
total_area_ml = []
lat = []
lon = []
 
table = soup.find("table", { "class" : "wikitable sortable" })
 
for row in table.findAll("tr"):
    cells = row.findAll("td")
    #For each "tr", assign each "td" to a variable.
    if len(cells) == 14:
        sort.append(cells[0].find(text=True))
        state.append(cells[1].find(text=True))
        FIPS.append(cells[2].find(text=True))
        county.append(cells[3].find(text=True))
        county_seat.append(cells[4].find(text=True))
        population.append(cells[5].find(text=True))
        land_area_km.append(cells[6].find(text=True))
        land_area_ml.append(cells[7].find(text=True))
        water_area_km.append(cells[8].find(text=True))
        water_area_ml.append(cells[9].find(text=True))
        total_area_km.append(cells[10].find(text=True))
        total_area_ml.append(cells[11].find(text=True))
        lat.append(cells[12].find(text=True))
        lon.append(cells[13].find(text=True))

cord = [('county', county[186:244]),
         ('lat', lat[186:244]),
         ('lon', lon[186:244]),
         ]
center_cord = pd.DataFrame.from_items(cord)

all_data_sub_cen = pd.merge(all_data_sub, center_cord, left_on='RegionName', right_on='county')
all_data_sub_cen["lon"] = ['-'+i[1:9] for i in all_data_sub_cen["lon"]]
all_data_sub_cen["lat"] = [i[1:8] for i in all_data_sub_cen["lat"]]

all_data_sub_cen.head()

,RegionName,sold_15,sold_16,list_15,list_16,Score2016,Score2015,coordinates,color,county,lat,lon
0,Los Angeles,498737.500000,518508.333333,537666.666667,587328.666667,0.575243,0.569822,"{u'geometry': {u'type': u'MultiPolygon', u'coo...",#fe916b,Los Angeles,34.1963,-118.2618
1,Orange,601020.833333,623150.000000,675932.083333,705890.000000,0.674438,0.682548,"{u'geometry': {u'type': u'MultiPolygon', u'coo...",#f67862,Orange,33.6756,-117.7772
2,San Diego,472187.500000,489250.000000,538650.000000,575525.000000,0.617286,0.625467,"{u'geometry': {u'type': u'MultiPolygon', u'coo...",#ff986e,San Diego,33.0236,-116.7761
3,Riverside,309068.750000,320437.500000,344257.500000,361732.458333,0.410757,0.411668,"{u'geometry': {u'type': u'MultiPolygon', u'coo...",#ffbf86,Riverside,33.7298,-116.0022
4,San Bernardino,264327.083333,272666.666667,271312.500000,291733.333333,0.434529,0.442580,"{u'geometry': {u'type': u'MultiPolygon', u'coo...",#ffc88d,San Bernardino,34.8572,-116.1811


In [183]:
mapbox_access_token = 'pk.eyJ1IjoiZGFuaWVsbGU5MTUxNSIsImEiOiJjaXpyeW44cTYwMGtvMzJsOXZtNTEwNmJtIn0.8d-m0sZPWnlRzH-T4OlAiA'

In [186]:
layers_ls = []
for x in all_data_sub.index:
    item_dict = dict(sourcetype = 'geojson',
                     source = all_data_sub.ix[x]['coordinates'],
                     type = 'fill',
                     color = all_data_sub.ix[x]['color'])
    layers_ls.append(item_dict)
test = []
for i in range(len(all_data_sub_cen["RegionName"].tolist())):
    test.append(all_data_sub_cen["RegionName"].tolist()[i] + '</br>' + '2015 Solde Price: ' + str(all_data_sub_cen["sold_15"].tolist()[i]))

In [187]:
data = go.Data([
        go.Scattermapbox(
            lat=all_data_sub_cen["lat"].tolist(),
            lon=all_data_sub_cen["lon"].tolist(),
            text=test,
            mode = 'markers',
                    marker = go.Marker(cmax=int(all_data_sub['sold_15'].max()/0.01),cmin=0,colorscale = colorscl,
                        showscale = True,autocolorscale=False,color=range(0,100),
                    colorbar= go.ColorBar(len = .89)
                                       ),
        )
    ])


layout = go.Layout(
    height=600,
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        layers=layers_ls,
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=39,
            lon=-124
        ),
        pitch=0,
        zoom=5,
        style='light'
               )
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='county-level-choropleths-python')

In [365]:
#plot for scatter matrix
crim_rate = ({"crimerate":all_data["CrimeCases"]/all_data["Population"]})
crim_rate_log = ({"logcrimerate":[math.log(i) for i in all_data["CrimeCases"]/all_data["Population"]]})
income_rate = ({"incomerate":all_data["total_income"]/all_data["Population"]})
population_log =  ({"logpopulation":[math.log(i) for i in all_data["Population"]]})
dataframe = all_data[['sold_15','Unemploy_Rate','Score2015']]

all_data_sub = pd.concat([dataframe,pd.DataFrame(crim_rate_log),
                          pd.DataFrame(population_log),pd.DataFrame(income_rate)],axis=1)

fig = ff.create_scatterplotmatrix(all_data_sub, diag='histogram',
                                  colormap='Blues', height=800, width=800)
py.iplot(fig, filename = 'Scatterplot Matrix')